In [ ]:
import numpy as np
from scipy import signal
import control as ct
import matplotlib.pyplot as plt

from spectools import lpsd, dsp
lpsd.use_c_core = False

import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()

In [ ]:
# Step 1: Define the system transfer function
tau = 0.1  # Time constant
fs = 1000  # Sampling frequency (Hz)
dt = 1 / fs  # Time step
n = int(1e6)  # Number of samples

# Transfer function H(s) = 1 / (tau*s + 1)
num = [1]
den = [tau, 1]
system = signal.TransferFunction(num, den)

# Step 2: Generate the input signal (white noise)
np.random.seed(42)  # For reproducibility
input_signal = np.random.normal(0, 1, n)

# Step 3: Generate the output signal by filtering the input through the system
t = np.linspace(0, n*dt, n)
_, output_signal, _ = signal.lsim(system, input_signal, t)

# Introduce frequency-dependent noise to the output
freq = np.fft.fftfreq(n, d=dt)
noise = np.fft.ifft(np.fft.fft(np.random.normal(0, 1, n)) * (np.abs(freq)**0.5)).real
output_signal_noisy = output_signal + 0.5 * noise  # Scale noise appropriately

In [ ]:
figsize=(5,3)
dpi=300
fontsize=8
linewidth=0.75

fig, ax = plt.subplots( figsize=figsize, dpi=dpi);
ax.plot(output_signal_noisy, linewidth=linewidth, color="brown", ls='-', label='output', alpha=0.5);
ax.plot(input_signal, linewidth=linewidth, color="black", label='input');
ax.set_xlabel("Time (s)", fontsize=fontsize);
ax.set_ylabel("Signals", fontsize=fontsize);
ax.tick_params(labelsize=fontsize);
ax.grid();
ax.legend(fontsize=fontsize);
ax.set_xlim(0,1000);
fig.tight_layout();
fig.align_ylabels();
plt.show()

In [ ]:
# Step 4: Compute the cross-spectral density (Welch)
f, Pxy = signal.csd(input_signal, output_signal_noisy, fs=fs, nperseg=4000)

In [ ]:
# Step 5: Compute the cross-spectral density (LTF)
ltf_obj = lpsd.ltf([input_signal, output_signal_noisy], fs=fs, pool=pool)

In [ ]:
# Step 6: Compute the auto-spectral den
auto_input = lpsd.ltf(input_signal, fs=fs, pool=pool)
auto_output = lpsd.ltf(output_signal_noisy, fs=fs, pool=pool)

In [ ]:
figsize=(5,3)
dpi=300
fontsize=8
linewidth=1.5

fig, (ax1, ax2) = plt.subplots(2,1, figsize=figsize, dpi=dpi);
ax1.semilogx(f, ct.mag2db(np.abs(Pxy)), linewidth=linewidth, color="orange", ls='--', label='Welch');
ax2.semilogx(f, -np.angle(Pxy, deg=True), linewidth=linewidth, color="orange", ls='--');
ax2.semilogx(ltf_obj.f, np.angle(ltf_obj.Gxy, deg=True), linewidth=linewidth, color="black");
ax1.semilogx(ltf_obj.f, ct.mag2db(np.abs(ltf_obj.Gxy)), linewidth=linewidth, color="black", label='LTF');
ax2.set_xlabel("Frequency (Hz)", fontsize=fontsize);
ax1.set_ylabel("Magnitude (dB)", fontsize=fontsize);
ax2.set_ylabel("Phase (deg)", fontsize=fontsize);
ax1.set_title("Cross spectral density estimate", fontsize=fontsize);
ax1.tick_params(labelsize=fontsize);
ax2.tick_params(labelsize=fontsize);
ax1.grid();
ax2.grid();
ax1.legend(fontsize=fontsize);
ax1.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax2.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax1.axes.xaxis.set_ticklabels([]);
fig.tight_layout();
fig.align_ylabels();
plt.show()

In [ ]:
figsize=(5,3)
dpi=300
fontsize=8
linewidth=1.5

fig, (ax1, ax2) = plt.subplots(2,1, figsize=figsize, dpi=dpi);

ax1.loglog(ltf_obj.f, np.abs(ltf_obj.Hxy), linewidth=linewidth, color="black");
ax1.fill_between(ltf_obj.f, 
                 np.abs(ltf_obj.Hxy) - 3*ltf_obj.Hxy_dev,
                 np.abs(ltf_obj.Hxy) + 3*ltf_obj.Hxy_dev,
                 color="lightblue", label=r"$\pm 3\sigma$")

ax2.semilogx(ltf_obj.f, np.angle(ltf_obj.Hxy), linewidth=linewidth, color="black");
ax2.fill_between(ltf_obj.f, 
                 np.angle(ltf_obj.Hxy)*(1 - 3*ltf_obj.Hxy_angle_error),
                 np.angle(ltf_obj.Hxy)*(1 + 3*ltf_obj.Hxy_angle_error),
                 color="lightblue")

ax2.set_xlabel("Frequency (Hz)", fontsize=fontsize);
ax1.set_ylabel("Magnitude (units)", fontsize=fontsize);
ax2.set_ylabel("Phase (rad)", fontsize=fontsize);
ax1.set_title(r"Transfer function estimate, $H_{xy}(f)$", fontsize=fontsize);
ax1.tick_params(labelsize=fontsize);
ax2.tick_params(labelsize=fontsize);
ax1.grid();
ax2.grid();
ax1.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax2.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax1.axes.xaxis.set_ticklabels([]);
ax2.set_ylim(-6,6)
ax1.legend(fontsize=fontsize);
fig.tight_layout();
fig.align_ylabels();
plt.show()

In [ ]:
figsize=(6,4)
dpi=300
fontsize=8
linewidth=1.5

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.loglog(ltf_obj.f, ltf_obj.Hxy_dev, linewidth=linewidth, label="Standard deviation", color="orange")
ax.loglog(ltf_obj.f, np.abs(ltf_obj.Hxy)*ltf_obj.Hxy_magnitude_error, linewidth=linewidth, color="tomato", ls='--')

ax.set_xlim([ltf_obj.f[0],ltf_obj.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize)
ax.set_ylabel(r"ASD $\rm (A/Hz^{1/2})$", fontsize=fontsize)
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid(which='both')
# ax.set_title("")
# ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
fig.tight_layout()
plt.show()

In [ ]:
figsize=(5,3)
dpi=300
fontsize=8
linewidth=1.5

fig, ax = plt.subplots( figsize=figsize, dpi=dpi);
ax.semilogx(ltf_obj.f, ltf_obj.coh, linewidth=linewidth, color="black", label='Coherence');
ax.semilogx(ltf_obj.f, ltf_obj.coh_error, linewidth=linewidth, color="brown", ls='-', label='Normalized random error');
ax.semilogx(ltf_obj.f, ltf_obj.coh_dev, linewidth=linewidth, color="orange", ls='--', label='Standard deviation');
ax.semilogx(ltf_obj.f, ltf_obj.coh_error*ltf_obj.coh, linewidth=linewidth, color="tomato", ls=':');
ax.set_xlabel("Frequency (Hz)", fontsize=fontsize);
ax.set_ylabel("Coherence", fontsize=fontsize);
ax.tick_params(labelsize=fontsize);
ax.grid();
ax.legend(fontsize=fontsize);
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax.set_ylim(0,1.2)
fig.tight_layout();
fig.align_ylabels();
plt.show()

In [ ]:
figsize=(5,3)
dpi=300
fontsize=8
linewidth=1.5

fig, ax = plt.subplots( figsize=figsize, dpi=dpi);
ax.semilogx(ltf_obj.f, ltf_obj.coh_dev, linewidth=linewidth, color="orange", ls='--', label='Standard deviation');
ax.semilogx(ltf_obj.f, ltf_obj.coh_error*ltf_obj.coh, linewidth=linewidth, color="tomato", ls=':');
ax.set_xlabel("Frequency (Hz)", fontsize=fontsize);
ax.set_ylabel("Coherence", fontsize=fontsize);
ax.tick_params(labelsize=fontsize);
ax.grid();
ax.legend(fontsize=fontsize);
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax.set_ylim(0,0.04)
fig.tight_layout();
fig.align_ylabels();
plt.show()

In [ ]:
figsize=(5,3)
dpi=300
fontsize=8
linewidth=1.5

fig, ax = plt.subplots( figsize=figsize, dpi=dpi);

ax.semilogx(ltf_obj.f, ltf_obj.coh, linewidth=linewidth, color="black", label='Coherence');
ax.fill_between(ltf_obj.f, 
                 ltf_obj.coh - 3*ltf_obj.coh_dev,
                 ltf_obj.coh + 3*ltf_obj.coh_dev,
                 color="lightblue", label=r'$\pm 3\sigma$')

ax.set_xlabel("Frequency (Hz)", fontsize=fontsize);
ax.set_ylabel("Coherence", fontsize=fontsize);
ax.tick_params(labelsize=fontsize);
ax.grid();
ax.legend(fontsize=fontsize);
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1]);
ax.set_ylim(0,1.2)
fig.tight_layout();
fig.align_ylabels();
plt.show()